# SECTION 0: DATA CLEANING OVERVIEW

Τhis section is responsible for the data cleaning process needed in order to have a valid data set for our analysis in the next sections. Prior to this step we had already run a set of SQL queries in order to validate:

1. **Null counts** on critical columns (Order Date, Region, Category, Profit, Discount)
2. **Invalid discount values** outside the [0, 1] range 
3. **Potential duplicates** at the (Order ID, Customer ID, Product ID) level
4. **Distinct categorical values** to check for spelling or formatting issues

The SQL script is stored in: `sql/superstore_data_integrity_checks.sql`

**⇒** Below we present all the steps of the Python data cleaning pipeline:






---
### i) Datetime Conversion
- Converts `order_date`, `ship_date` to proper datetime format.
- Coerces invalid strings to NaT (Not a Time, i.e.missing value for dates).

In [5]:
for c in ["order_date","ship_date"]:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")


---
### ii) Numeric Fields
- Ensures numeric columns (`sales`, `profit`, `discount`) are floats/ints.
- Coerces invalid entries to NaN.


In [7]:
for c in ["sales","profit","discount","quantity","postal_code"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

---
### iii) Boundary Rules
- Flags absurd values such as negative sales and unrealisticly profit/loss.

In [13]:
print("Rows with sales <= 0:", int((df['sales'] <= 0).sum()))
print("Rows with |profit| > 100000:", int((df['profit'].abs() > 100000).sum()))

Rows with sales <= 0: 0
Rows with |profit| > 100000: 0


---
### iv) Column Normalization
- Normalize columns names to drop capital letters etc and have all data names in a unified way

In [12]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("-", "_")
)

print("Dataset Column Names: ", df.columns.tolist())

Dataset Column Names:  ['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales', 'quantity', 'discount', 'profit']


---
### v) Clean Dataset Exportation
- Export the clean dataset in a pro in order to run the next data analysis steps

In [14]:
df_clean = df.copy()
df_clean.to_pickle("clean_superstore.pkl")